In [1]:
import z3
from pyspec import *

In [2]:
def mk_atoms(names: str) -> Atom:
    return (Atom(name.strip()) for name in names.split(','))

In [4]:
x,y,z = mk_atoms('x,y,z')


def eval_expr(es):
    print(es+': ', str(eval(es)))

examples = [
    'x + y + z',
    'x + (y+z)',
    'x < x << z',
    'x > x << z',
    'x << y << z',
    'x-y << z',
    'y < z',
    'x+y+x+y'
]

for e in examples:
    eval_expr(e)


x + y + z:  x+y+z
x + (y+z):  x+(y+z)
x < x << z:  x<(x<<z)
x > x << z:  x>(x<<z)
x << y << z:  x<<y<<z
x-y << z:  x-y<<z
y < z:  y<z
x+y+x+y:  x+y+x+y


In [69]:
z3_A = z3.DeclareSort('A')



class A:

    def __init__(self, *args):
        self.args = args
        self.op = None
        self.assoc = True


    def __str__(self):
        if self.op is None:
            return str(self.args[0] if len(self.args)==1 else self.args)
        else:
            args = [str(x) 
                    if (x.op is None)
                    or (self.__class__ != x.__class__ and issubclass(self.__class__, x.__class__))
                    or (self.op == x.op and self.assoc)
                    else f'({str(x)})' for x in self.args]
            return self.op.join(args)



    def mk_operator(self, other, op):
        if isinstance(other, A):
            return op(self, other)
        return NotImplemented
    
    def __pow__(self, other):
        self.mk_operator(other, Power)

    def __add__(self, other):
        return self.mk_operator(other, Add)
    
    def __sub__(self, other):
        return self.mk_operator(self, other, Sub)

    def __mul__(self, other):
        return self.mk_operator(other, Mul)
    
    def __matmul__(self, other):
        return self.mk_operator(other, MatMul)

    def __truediv__(self, other):
        return self.mk_operator(other, TrueDiv)

    def __floordiv__(self, other):
        return self.mk_operator(other, FloorDiv)


class Power(A): pass

class Mul(A):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = True
        self.op = '*'

class MatMul(Mul):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = True
        self.op = '@'

class TrueDiv(MatMul):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = False
        self.op = '/'

class FloorDiv(TrueDiv):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = False
        self.op = '//'

class Add(FloorDiv):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = True
        self.op = '+'

class Sub(Add):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = False
        self.op = '-'

class Pow(Sub):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = False
        self.op = '**'

class And(Pow):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = True
        self.op = '&'

class Xor(And):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = True
        self.op = '^'

class Or(Xor):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.assoc = True
        self.op = '|'




    
a0 = A('a0')
a1 = A('a1')
a2 = A('a2')

print((a0 + a1) + a2)
print(a0 + (a1 + a2))
print((a0 - a1) + a2)
print(a0 - (a1 + a2))


print((a0 / a1) * a2)
print((a0 * a1) + a2)
print(a0 + (a1 * a2))
print(a0 + (a1 / a2))
print(a0 - (a1 - a2))
print(a0 - a1 - a2)
print(a0 + (a1 + a2))
print(a0 * a1 * a2)
print(a0 / a1 / a2)
print((a0 / a1) / a2)
print(a0 / (a1 / a2))



a0+a1+a2
a0+a1+a2
(a0-a1)+a2
a0-a1+a2
(a0/a1)*a2
a0*a1+a2
a0+a1*a2
a0+a1/a2
a0-(a1-a2)
(a0-a1)-a2
a0+a1+a2
a0*a1*a2
(a0/a1)/a2
(a0/a1)/a2
a0/(a1/a2)


In [74]:
2 ** (3 ** 4)

2417851639229258349412352